In [178]:
import numpy  as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder, FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


In [179]:
from sklearn.compose import ColumnTransformer

In [180]:
train_ds=pd.read_csv('./train.csv')
test_ds=pd.read_csv('./test.csv')

In [181]:
train_ds=train_ds.drop(columns=['Name','Ticket','PassengerId','Cabin'],axis=1)
test_ds=test_ds.drop(columns=['Name','Ticket','PassengerId','Cabin'],axis=1)

In [182]:
Numeric_cols=train_ds.select_dtypes(include=['number']).columns.tolist()
categorical_cols=train_ds.select_dtypes(include=['object','category']).columns.tolist()


In [183]:
for col in Numeric_cols:
    print(col,"(Nan count)=>",train_ds[col].isna().sum())

Survived (Nan count)=> 0
Pclass (Nan count)=> 0
Age (Nan count)=> 177
SibSp (Nan count)=> 0
Parch (Nan count)=> 0
Fare (Nan count)=> 0


In [184]:
for col in categorical_cols:
    print(col,"(Nan count)=>",train_ds[col].isna().sum(),train_ds[col].value_counts())

Sex (Nan count)=> 0 Sex
male      577
female    314
Name: count, dtype: int64
Embarked (Nan count)=> 2 Embarked
S    644
C    168
Q     77
Name: count, dtype: int64


## **Train Test Split**

In [185]:
from sklearn.model_selection import train_test_split

In [186]:
target= train_ds['Survived']
train_ds=train_ds.drop(columns='Survived', axis=1)
X_train, X_test, y_train, y_test=train_test_split(train_ds, target, test_size=0.2, random_state=42)

In [187]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((712, 7), (712,), (179, 7), (179,))

## **Pipeline**

In [188]:
from  sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2

In [189]:
X_train.info(),y_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 331 to 102
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       572 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  710 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 44.5+ KB
<class 'pandas.core.series.Series'>
Index: 712 entries, 331 to 102
Series name: Survived
Non-Null Count  Dtype
--------------  -----
712 non-null    int64
dtypes: int64(1)
memory usage: 11.1 KB


(None, None)

In [190]:
#fill nan in Age_col
trf1=ColumnTransformer([
    ('impute_age',SimpleImputer(strategy='mean'),[2]),
    ('impute_Emberked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [191]:
#sex_OHE
trf2=ColumnTransformer([
    ('sex_encoder',OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[1])
],remainder='passthrough')

In [192]:
#sex_OHE
trf3=ColumnTransformer([
    ('Emberked_encoder',OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[6])
],remainder='passthrough')

In [193]:
#scaling
trf4=ColumnTransformer([
    ('scaler',MinMaxScaler(),slice(0,10))
])

In [194]:
#feature
trf5=SelectKBest(score_func=chi2, k=7)

In [195]:
#model
#trf6=DecisionTreeClassifier()
trf6=RandomForestClassifier()

pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [196]:
pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5,trf6)

In [197]:
#X_train_df = pd.DataFrame(X_train,columns=train_ds.columns)
#y_train_df = pd.DataFrame(y_train, columns=train_ds.columns)

pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_Emberked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('sex_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),...
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Emberked_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [6])])),
                ('columntransformer-4',
                 ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=7,
                             score_func=<function chi2 at 0x000001777678BC40>)),
                ('randomforestclassifier', RandomForestClassifier())])

In [198]:
from sklearn import set_config
set_config(display='diagram')

In [199]:
y_pred=pipe.predict(X_test)

In [200]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6480446927374302

In [202]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Step 4: Instantiate GridSearchCV
grid_search = GridSearchCV(pipe, param_grid, cv=5)

# Step 5: Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Step 6: Access results
print("Best parameters found: ", grid_search.best_params_)
print("Best score found: ", grid_search.best_score_)

ValueError: Invalid parameter 'max_depth' for estimator Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_Emberked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('sex_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),...
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Emberked_encoder',
               

In [ ]:
X_test.shape

In [ ]:
train_ds.head()

In [ ]:
import pickle
pickle.dump(pipe, open('pipe_dt.pkl','wb'))